In [301]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string 
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from nltk.translate.bleu_score import sentence_bleu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

%matplotlib inline 
## mathplot 그래프를 출력 영역에 표시


In [105]:
import zipfile
zipfile.ZipFile("data.zip").extractall()

In [106]:
## 단어 stoi, itos화

SOS_token = 0
EOS_token = 1

class Lang:
    
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1:"EOS"}
        self.n_words = 2
    
    def addSentence(self, sentence):
        for word in sentence.split(" "):
            self.addWord(word)
            
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [107]:
#유니 코드 문자열을 일반 ASCII로 변환

def unicodeToAscii(s):
    return ''.join(
    c for c in unicodedata.normalize('NFD',s) 
    if unicodedata.category(c) != 'Mn' )

##소문자로 만들기, 문자 아닌 문자 제거.

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip()) ## strip(), 양 끝의 공백을 지움
    s = re.sub(r"([.!?])", r" \1", s) ##지우는게 아니라 한칸 띄어쓰기가 적용됨
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [108]:
def readLang(lang1, lang2, reverse=False):
    print("Read lines...")
    
    ##파일을 읽고 줄로 분리
    lines = open(f'data/{lang1}-{lang2}.txt', encoding='utf-8').read().strip().split('\n')
    
    ##모든 줄을 쌍으로 분리하고 정규화
    pairs =[[normalizeString(s) for s in l.split('\t')] for l in lines]
    
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
    
    return input_lang, output_lang, pairs


    lines = open(f'data/eng-fra.txt', encoding='utf-8').read()
     = 'Go.\tVa !\nRun!\tCours\u202f!\nRun!\tCourez\u202f!\nWow!\tÇa alors\u202f'

    lines = open(f'data/eng-fra.txt', encoding='utf-8').read().strip().split('\n')
     = ['Go.\tVa !',
     'Run!\tCours\u202f!',
     'Run!\tCourez\u202f!',
     'Wow!\tÇa alors\u202f!',
     'Fire!\tAu feu !',
     "Help!\tÀ l'aide\u202f!",
     'Jump.\tSaute.',
     'Stop!\tÇa suffit\u202f!',
     'Stop!\tStop\u202f!',
     'Stop!\tArrête-toi !']

     pairs= [[normalizeString(s) for s in l.split('\t')] for l in lines]
      = [['go .', 'va !'],
     ['run !', 'cours !'],
     ['run !', 'courez !'],
     ['wow !', 'ca alors !'],
     ['fire !', 'au feu !'],
     ['help !', 'a l aide !'],
     ['jump .', 'saute .'],
     ['stop !', 'ca suffit !'],
     ['stop !', 'stop !'],
     ['stop !', 'arrete toi !']]

In [109]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

In [110]:
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes) ##eng_prefixes에만 있는 단어 True, False 반환, 결과적으로 True만 반환

##10글자 미만 시작이 eng_prefixes로 시작하는 문장들만.

In [273]:
len(pairs)

10599

    pairs[0][1].split(' ')
     = ['va', '!']

    print(pairs[9]) 
    a = filterPair(pairs[9])
    print(a)
    =['stop !', 'arrete toi !']
     False

In [111]:
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [317]:
##파일을 읽고 줄로 분리, 줄을 쌍으로 분리
## 텍스트 정규화하고 길이,내용으로 필터링 , pre_fix filtering
## 쌍을 이룬 문장들로 단어 리스트 생성

def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLang(lang1, lang2, reverse)
    
    print("Read %s sentence pairs" % len(pairs))
    #pairs = filterPairs(pairs)
    
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    
    return input_lang, output_lang, pairs



In [318]:
##lnput_lang 단어 사전
input_lang, output_lang, pairs = prepareData('eng','fra', True)
print(random.choice(pairs))

Read lines...
Read 135842 sentence pairs
Trimmed to 135842 sentence pairs
Counting words...
Counted words:
fra 21334
eng 13043
['qu as tu d autre dans la poche ?', 'what else do you have in your pocket ?']


In [319]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(input_size , hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size) ##hidden_size 크기를 받아서 hidden_size로 출력
    
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1,1,-1)
        output = embedded ##(1, 1, hidden_size)
        output, hidden = self.gru(output, hidden) #(1,1,hidden), (hidden)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1,1,self.hidden_size, device = device)

In [320]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim = 1)
    
    def forward(self, inputs, hidden):
        output = self.embedding(inputs).view(1,1,-1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        a = self.softmax(self.out(output[0]))
        output = self.softmax(self.out(output[0]))
        return output, hidden
    def initHidden(self):
        return torch.zeros(1,1,self.hidden_size , device =device)

In [321]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype = torch.long, device = device).view(-1,1)

def tensorFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [346]:
def lenFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)


for i in pairs:
    c, b = 0
    a = lenFromPair(i)
    if a[]

TypeError: 'int' object is not iterable

In [353]:
len(pairs[135841][0])

348

In [359]:
teacher_forcing_ratio = 0.5
## batch_first = True (N, L, H ) // batch_first = False (L, N, H) 
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length = MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    #encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device = device)
    
    loss = 0
    
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei],encoder_hidden)
        #encoder_outputs[ei] = encoder_output[0,0] ##0번째 시퀀스, 0번째 배치의, 히든 사이즈 attention 용
    
    decoder_input = torch.tensor([[SOS_token]], device= device)
    
    decoder_hidden = encoder_hidden
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        ## 교사학습이 이뤄지면 타겟단어를 다음 입력으로 전달
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden) #, encoder_outputs
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di] #teacher forcing
    
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)#, encoder_outputs
            
            topv, topi = decoder_output.topk(1) ##각 배치의 각 시퀀스 가장 큰 수 하나를 뽑음, 그 값과 인덱스를 반환함. 
            decoder_input = topi.squeeze().detach() ##입력으로 사용할 부분을 히스토리에서 분리 그래디언트 계산 해제, 입력값 이기 때문에.
            
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break
    
    loss.backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item() / target_length
            

In [119]:
import time
import math

def asMinutes(s):
    m = math.floor(s/60)
    s -= m*60
    
    return f'{m}m {s}s'


def timeSince(since, percent):
    now = time.time()
    s = now - since
    
    es = s / (percent)
    
    rs = es - s
    
    return f"{asMinutes(s)} (- {asMinutes(rs)})"

In [325]:
def trainIters(encoder, decoder, n_iters, print_every = 1000, plot_every = 100, learning_rate = 0.001):
    start = time.time()
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0
    
    encoder_optimizer = optim.Adam(encoder.parameters(), lr = learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr = learning_rate)
    
    training_pairs = [tensorFromPair(random.choice(pairs)) for i in range(n_iters)] ## list 안에 (source, target) 형태로 생성
    criterion = nn.CrossEntropyLoss()
    
    for iter in range(1, n_iters+1):
        training_pair = training_pairs[iter-1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        
        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
    
        print_loss_total += loss
        plot_loss_total += loss
    
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                                     iter, iter / n_iters * 100, print_loss_avg))        
        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
        
    showPlot(plot_losses)
            
        

In [326]:
 training_pairs = [tensorFromPair(random.choice(pairs)) for i in range(1)]

In [327]:
training_pairs

[(tensor([[138],
          [119],
          [604],
          [ 69],
          [198],
          [  3],
          [  1]], device='cuda:0'),
  tensor([[ 188],
          [ 189],
          [ 357],
          [1145],
          [  94],
          [2926],
          [   3],
          [   1]], device='cuda:0'))]

In [328]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base= 0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [356]:
def evaluate(encoder, decoder, sentence, max_length = MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        
        #encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device = device)
        
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            #encoder_outputs[ei] = encoder_output[0,0]
            
        decoder_input = torch.tensor([[SOS_token]], device = device)
        
        decoder_hidden = encoder_hidden
        
        decoded_word = []
        
        for di in range(max_length): ## 1,1, hidden 값이 들어감
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden) #if attention, encoder_outputs 추가
            topv, topi = decoder_output.topk(1)
            if topi.item() == EOS_token:
                decoded_word.append("<EOS>")
                break
            else:
                decoded_word.append(output_lang.index2word[topi.item()])
                
            decoder_input = topi.squeeze().detach()
            
            
        
        return decoded_word
            
                
            
        
        

## 학습 실행

In [360]:
hidden_size = 256
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder, decoder, 75000, print_every= 5000)

KeyboardInterrupt: 

In [264]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [294]:
output_words = evaluate(encoder, decoder, pair[0])

In [300]:
print(type(output_words))
print(type(pair[1].split()))

<class 'list'>
<class 'list'>


In [314]:
def evaluate_BLEU(encoder, decoder):
    total_bleu = 0
    for i in range(len(pairs)):
        pair = pairs[i]
        output_words = evaluate(encoder, decoder, pair[0])
        real_words = [pair[1].split()]
        bleu_score = sentence_bleu(real_words, output_words, weights = (0.5,0.5,0,0))
        total_bleu += bleu_score
    print(f"mean BLEU score : {total_bleu / len(pairs)}")

In [316]:
only_decoder_encoder_BLEU = evaluate_BLEU(encoder, decoder)

mean BLEU score : 0.6727973615482881


In [265]:
evaluateRandomly(encoder, decoder)

> je ne suis pas medecin mais professeur .
= i am not a doctor but a teacher .
< i m not a doctor but teacher . <EOS>

> nous sommes censes etre ensemble .
= we re meant to be together .
< we re meant to be together . <EOS>

> il est sur le point de partir .
= he s about to go .
< he s about to leave . <EOS>

> je ne suis pas plus intelligent que lui .
= i m not as intelligent as he is .
< i m not as smart as he . <EOS>

> j agis pour mon pere .
= i am acting for my father .
< i am acting for my father . <EOS>

> tu preches un convaincu .
= you re preaching to the choir .
< you re preaching to the choir . <EOS>

> tu es tres genereuse .
= you re very generous .
< you re very generous . <EOS>

> elle n est pas d humeur .
= she s not in the mood .
< she s not in the mood . <EOS>

> j ai faim et soif .
= i m hungry and thirsty .
< i m thirsty and thirsty . <EOS>

> tu es tres brave .
= you re very brave .
< you re very brave . <EOS>

